<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget 'https://raw.githubusercontent.com/logpai/logparser/master/logs/HDFS/HDFS_2k.log'

--2023-08-31 09:25:40--  https://raw.githubusercontent.com/logpai/logparser/master/logs/HDFS/HDFS_2k.log
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 287848 (281K) [text/plain]
Saving to: ‘HDFS_2k.log’

HDFS_2k.log         100%[===================>] 281.10K  --.-KB/s    in 0.04s   

2023-08-31 09:25:40 (7.32 MB/s) - ‘HDFS_2k.log’ saved [287848/287848]



In [3]:
import re
import string

def clean(s):
    """ Preprocess log message
    Parameters
    ----------
    s: str, raw log message

    Returns
    -------
    str, preprocessed log message without number tokens and special characters
    """
    # s = re.sub(r'(\d+\.){3}\d+(:\d+)?', " ", s)
    # s = re.sub(r'(\/.*?\.[\S:]+)', ' ', s)
    s = re.sub('\]|\[|\)|\(|\=|\,|\;', ' ', s)
    s = " ".join([word.lower() if word.isupper() else word for word in s.strip().split()])
    s = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', s))
    s = " ".join([word for word in s.split() if not bool(re.search(r'\d', word))])
    trantab = str.maketrans(dict.fromkeys(list(string.punctuation)))
    content = s.translate(trantab)
    s = " ".join([word.lower().strip() for word in content.strip().split()])
    return s

In [11]:
text1 = '081109 203615 148 INFO dfs.DataNode$PacketResponder: PacketResponder 1 for block blk_38865049064139660 terminating'
cleaned_text1 = clean(text1)
print(text1)
print(cleaned_text1)

081109 203615 148 INFO dfs.DataNode$PacketResponder: PacketResponder 1 for block blk_38865049064139660 terminating
info dfs data node packet responder packet responder for block terminating


In [12]:
text2 = '081109 205632 28 INFO dfs.FSNamesystem: BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.251.74.79:50010 is added to blk_-4794867979917102672 size 67108864'
cleaned_text2 = clean(text2)
print(text2)
print(cleaned_text2)

081109 205632 28 INFO dfs.FSNamesystem: BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.251.74.79:50010 is added to blk_-4794867979917102672 size 67108864
info dfs fs namesystem block name systemadd stored block block map updated is added to size


In [50]:
import numpy as np


In [8]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=4b7873b1495350d914cde3d5bfa7e1a426b039c5c4570318b332f4865108d76e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [9]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')


In [13]:
embeddings1 = model.encode(cleaned_text1)
print(embeddings1.shape)

(384,)


In [14]:
embeddings2 = model.encode(cleaned_text2)
util.cos_sim(embeddings1, embeddings2)

tensor([[0.5635]])

In [15]:
with open('/content/HDFS_2k.log', "r") as file:
    for line in file:
      print(line)
      break

081109 203615 148 INFO dfs.DataNode$PacketResponder: PacketResponder 1 for block blk_38865049064139660 terminating



In [26]:
with open('/content/HDFS_2k.log', "r") as file:
    dic = {}
    for line in file:
      cleaned_line = clean(line)
      if dic.get(cleaned_line) is None:
          embeddings = model.encode(cleaned_line)
          dic[cleaned_line] = embeddings

In [29]:
len(dic)

14

In [38]:
list(dic.keys())[0]

'info dfs data node packet responder packet responder for block terminating'

In [46]:
dic[list(dic.keys())[0]]

array([-4.18840051e-02,  3.08060106e-02,  7.60024711e-02, -4.43329588e-02,
        6.42990470e-02, -1.57718807e-01,  3.11618652e-02, -1.33357598e-02,
       -2.62779035e-02,  9.68374088e-02, -1.72485225e-02,  7.69180572e-03,
       -9.42633674e-02,  2.11521965e-02, -2.83487327e-02,  4.24248651e-02,
        7.21048703e-03, -7.77668040e-03,  3.22379842e-02, -5.85394539e-03,
        3.59230414e-02,  1.05570741e-01,  1.59710143e-02,  5.53610139e-02,
       -5.54818958e-02,  2.36416105e-02, -4.22507562e-02, -7.19283661e-03,
        3.40293790e-03, -3.56121026e-02,  1.60273071e-02, -5.91035187e-03,
       -9.98105295e-03,  1.56616233e-03,  1.29514271e-02,  7.25036636e-02,
        1.58867743e-02,  8.78086761e-02, -9.29258913e-02,  4.35930863e-03,
        6.00217879e-02,  2.24389955e-02,  5.95181191e-04,  7.26612881e-02,
        4.39117774e-02,  5.63253686e-02, -6.07074723e-02,  3.36782560e-02,
        1.91955436e-02, -1.11455200e-02, -5.71915433e-02,  7.26076290e-02,
        5.43761114e-03,  

In [51]:
similarities = util.cos_sim(dic[list(dic.keys())[0]], np.array(list(dic.values())))

In [54]:
similarities[0]

tensor([1.0000, 0.5635, 0.8446, 0.7015, 0.5992, 0.6265, 0.6297, 0.7440, 0.5559,
        0.5444, 0.7458, 0.6072, 0.6655, 0.6764])

In [55]:
for i, similarity in enumerate(similarities[0]):
    print("Similarity:", similarity)
    print(list(dic.keys())[0])
    print(list(dic.keys())[i])
    print('------------------------------------')


Similarity: tensor(1.0000)
info dfs data node packet responder packet responder for block terminating
info dfs data node packet responder packet responder for block terminating
------------------------------------
Similarity: tensor(0.5635)
info dfs data node packet responder packet responder for block terminating
info dfs fs namesystem block name systemadd stored block block map updated is added to size
------------------------------------
Similarity: tensor(0.8446)
info dfs data node packet responder packet responder for block terminating
info dfs data node packet responder received block of size from
------------------------------------
Similarity: tensor(0.7015)
info dfs data node packet responder packet responder for block terminating
info dfs data node data xceiver receiving block src dest
------------------------------------
Similarity: tensor(0.5992)
info dfs data node packet responder packet responder for block terminating
info dfs fs namesystem block name systemallocate block

In [62]:
similarities = util.cos_sim(np.array(list(dic.values())), np.array(list(dic.values())))
similarities

tensor([[1.0000, 0.5635, 0.8446, 0.7015, 0.5992, 0.6265, 0.6297, 0.7440, 0.5559,
         0.5444, 0.7458, 0.6072, 0.6655, 0.6764],
        [0.5635, 1.0000, 0.6606, 0.5517, 0.8255, 0.5742, 0.6767, 0.5626, 0.3707,
         0.7587, 0.5605, 0.7330, 0.6223, 0.6873],
        [0.8446, 0.6606, 1.0000, 0.7109, 0.5863, 0.6029, 0.6015, 0.7213, 0.5287,
         0.4788, 0.6790, 0.5728, 0.8092, 0.6526],
        [0.7015, 0.5517, 0.7109, 1.0000, 0.5901, 0.6355, 0.6331, 0.9244, 0.7144,
         0.5104, 0.6724, 0.5595, 0.9316, 0.6820],
        [0.5992, 0.8255, 0.5863, 0.5901, 1.0000, 0.5767, 0.6823, 0.6063, 0.3678,
         0.8474, 0.6107, 0.8242, 0.5765, 0.7891],
        [0.6265, 0.5742, 0.6029, 0.6355, 0.5767, 1.0000, 0.6342, 0.6362, 0.4870,
         0.5566, 0.5771, 0.5751, 0.6015, 0.6597],
        [0.6297, 0.6767, 0.6015, 0.6331, 0.6823, 0.6342, 1.0000, 0.6235, 0.4267,
         0.7476, 0.6167, 0.8413, 0.6192, 0.6678],
        [0.7440, 0.5626, 0.7213, 0.9244, 0.6063, 0.6362, 0.6235, 1.0000, 0.7487,
  

In [64]:

# Compute pairwise similarities

similarities = np.triu(similarities, k=1)  # Exclude diagonal and lower triangular elements, change them into zero

# Find indices where similarity exceeds threshold
indices = np.where(similarities > 0.8)

# Iterate over similar template pairs
for i, j in zip(indices[0], indices[1]):
    similarity = similarities[i, j]
    print('similarity =', similarity)
    print('template', i, ':', list(dic.keys())[i])
    print('template', j, ':', list(dic.keys())[j])
    print('--------------------------------')

similarity = 0.8445556
template 0 : info dfs data node packet responder packet responder for block terminating
template 2 : info dfs data node packet responder received block of size from
--------------------------------
similarity = 0.8254899
template 1 : info dfs fs namesystem block name systemadd stored block block map updated is added to size
template 4 : info dfs fs namesystem block name systemallocate block
--------------------------------
similarity = 0.8092111
template 2 : info dfs data node packet responder received block of size from
template 12 : info dfs data node data xceiver received block src dest of size
--------------------------------
similarity = 0.9244056
template 3 : info dfs data node data xceiver receiving block src dest
template 7 : info dfs data node data xceiver served block to
--------------------------------
similarity = 0.93164515
template 3 : info dfs data node data xceiver receiving block src dest
template 12 : info dfs data node data xceiver received blo

In [68]:
np.array(list(dic.values())).shape

(14, 384)

In [65]:
from sklearn.decomposition import PCA


def dim_reduction(vectors, n_components=20):
    estimator = PCA(n_components)
    pca_result = estimator.fit_transform(vectors)
    print('the percentage of variance for 20D : ', sum(estimator.explained_variance_ratio_ * 100))

    # Step1-3 PPA: De-averaged
    ppa_result = []
    result = pca_result - np.mean(pca_result)
    pca = PCA(n_components=20)
    pca_result = pca.fit_transform(result)
    U = pca.components_
    for i, x in enumerate(result):
        for u in U[0:7]:
            x = x - np.dot(u.transpose(), x) * u
        ppa_result.append(list(x))
    ppa_result = np.array(ppa_result)

    return ppa_result


In [74]:
import numpy as np
from sklearn.decomposition import PCA

# Assume you have a dictionary named 'embeddings_dict' with texts as keys and embeddings as values

# Step 1: Prepare Data
standardized_embeddings = np.array(list(dic.values()))


# Step 3: Apply PCA
num_components = 14  # Choose the number of components you want
pca = PCA(n_components=num_components)
pca.fit(standardized_embeddings)

# Step 4: Transform Data
reduced_embeddings = pca.transform(standardized_embeddings)

# Step 5: Inverse Transform (Optional)
# original_embeddings = pca.inverse_transform(reduced_embeddings)


In [78]:
np.sum(pca.explained_variance_ratio_[:10])

0.96261144

In [88]:
print(reduced_embeddings[:,:10].shape)
print(reduced_embeddings[0,:10].shape)

(14, 10)
(10,)


In [89]:

reduced_dic = {}
for i,key in enumerate(dic.keys()):
  reduced_dic[key]=reduced_embeddings[i,:10]

In [90]:
similarities = util.cos_sim(reduced_dic[list(dic.keys())[0]], np.array(list(reduced_dic.values())))
for i, similarity in enumerate(similarities[0]):
    print("Similarity:", similarity)
    print(list(reduced_dic.keys())[0])
    print(list(reduced_dic.keys())[i])
    print('------------------------------------')

Similarity: tensor(1.0000)
info dfs data node packet responder packet responder for block terminating
info dfs data node packet responder packet responder for block terminating
------------------------------------
Similarity: tensor(-0.3431)
info dfs data node packet responder packet responder for block terminating
info dfs fs namesystem block name systemadd stored block block map updated is added to size
------------------------------------
Similarity: tensor(0.5680)
info dfs data node packet responder packet responder for block terminating
info dfs data node packet responder received block of size from
------------------------------------
Similarity: tensor(-0.0824)
info dfs data node packet responder packet responder for block terminating
info dfs data node data xceiver receiving block src dest
------------------------------------
Similarity: tensor(-0.3367)
info dfs data node packet responder packet responder for block terminating
info dfs fs namesystem block name systemallocate bl